In [ ]:
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from functions import get_timeseries_table
from keras import Input
from keras import backend as K
from keras import regularizers, optimizers
from keras.layers import Dense, Dropout, ConvLSTM2D, Conv1D, \
AveragePooling1D

from sklearn.model_selection import train_test_split

The below is thanks to [dokondr](https://stackoverflow.com/questions/45411902/how-to-use-f1-score-with-keras-model)

In [ ]:
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score 

In [ ]:
df = get_timeseries_table(prediction_window=135, 
                          binary_labels=True, 
                          one_hot_modules=True)
df

In [ ]:
X = df.drop(columns='final_result')
y = df['final_result']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=111, test_size=.1)
X_t, X_val, y_t, y_val = train_test_split(X_train, y_train, random_state=111, test_size=.1)

In [ ]:
l1 = 1e-4
l2 = 1e-3
bias = 1e-4
dropout500=0.4
dropout300=0.2
dropout200=0.2

model1 = keras.Sequential()
model1.add(Input((X.shape[1])))
model1.add(Dense(500, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout500))
model1.add(Dense(500, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout500))
model1.add(Dense(500, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dense(500, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout500))
model1.add(Dense(500, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout500))
model1.add(Dense(500, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout500))
model1.add(Dense(300, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout300))
model1.add(Dense(300, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout300))
model1.add(Dense(300, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout300))
model1.add(Dense(200, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout200))
model1.add(Dense(200, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout200))
model1.add(Dense(200, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout200))
model1.add(Dense(200, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout200))
model1.add(Dense(200, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))
model1.add(Dropout(rate=dropout200))
model1.add(Dense(200, activation='relu', 
                 kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                 bias_regularizer=regularizers.l2(bias)))

model1.add(Dense(1, activation='sigmoid'))

optimizer = optimizers.Adamax()

model1.compile(optimizer=optimizer, loss='binary_crossentropy',
              metrics=['accuracy'])

#class_balance = num(pass) / num(fail).  Use this to weight class 0.
class_balance = sum(y_t == 1) / sum(y_t == 0)

model1.fit(X_t, y_t,
           batch_size = 200,
           epochs = 50,
           validation_data=(X_val, y_val),
        #    class_weight = {1: 1.0,
        #                    0: class_balance}
           )